# Домашнее задание №3.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon
from scipy.stats import uniform
from scipy.stats import chi2
from scipy.stats import gamma
from scipy.stats import beta
import seaborn as sns
%config InlineBackend.figure_format = 'retina'

## ***Задача 1: распределение верхних экстремумов для показательной модели (2 балла)***

### **1.** Зафиксируйте $n$.

In [ ]:
n = 100

### **2.** Сгенерируйте $K = n^2$ выборок $(X_1^k, X_2^k, ... , X_n^k)$ объема $n$ из экспоненциального распределения $\operatorname{Exp}(1)$. В каждой из $K$ выборок определить максимум $X_{(n)}^k$, $1 \le k \le n$.

In [ ]:
K = n ** 2
lambda_ = 1
samples = []
sorted_samples = []
n_order_statistics = []
for i in range(K):
    samples.append(expon.rvs(scale=1 / lambda_, size=n))

for i in range(K):
    sorted_samples.append(sorted(samples[i]))
    n_order_statistics.append(sorted_samples[i][-1])

### **3.** Вычислите функцию \begin{equation}
    \hat G_K(x) = \cfrac{1}{K} \sum\limits_{k=1}^{K}\mathbb{1}\left(X_{(n)}^k - \ln n \le x \right), \;\;\; x \ge 0.
\end{equation} и постройте графики зависимостей $\hat G_K(x)$ и $G(x) = e^{-e^{-x}}$ от $x$.

In [ ]:
G_K_x = []
G_x = []
x = sorted(n_order_statistics)
b = np.log(n)

for i in range(len(x)):
    x[i] -= b

for i in range(len(x)):
    G_K_x.append(i / K)
    G_x.append(np.exp(-np.exp(-x[i])))

fig_1, ax_1 = plt.subplots(figsize=(9, 6))
plt.plot(x, G_K_x, color='green')
plt.title("график $\hat G_K(x)$")
plt.xlabel('x', fontsize=15)
plt.ylabel('$\hat G_K(x)$', fontsize=15)
plt.grid()

fig_2, ax_2 = plt.subplots(figsize=(9, 6))
plt.plot(x, G_x, color='red')
plt.title("график $\hat G(x)$")
plt.xlabel('x', fontsize=15)
plt.ylabel('$\hat G(x)$', fontsize=15)
plt.grid()

fig_3, ax_3 = plt.subplots(figsize=(9, 6))
plt.plot(x, G_x, color='red')
plt.plot(x, G_K_x, color='green')
plt.title("график $\hat G(x)$ и $\hat G(x)$")
plt.xlabel('x', fontsize=15)
plt.ylabel('$\hat G(x)$', fontsize=15)
plt.legend(labels=['$\hat G(x)$', '$\hat G_K(x)$'])
plt.grid()

plt.show()

### **4.** Убедитесь, что $\hat G_K$ близка к $G(x) = e^{-e^{-x}}$. Для этого вычислите \begin{equation}
    T_K = \operatorname{sup}\limits_{x \ge 0} \left| \hat G_K(x) - G(x) \right|.
\end{equation} Обратите внимание, что $\hat G_K$ -- кусочно постоянная функция, а G - монотонная. Поэтому значение статистики $T_K$ можно найти точно.

In [ ]:
T_k = 0
for i in range(len(x) - 1):
    T_k = max(T_k, abs(G_x[i] - G_K_x[i]), abs(G_x[i] - G_K_x[i + 1]))
T_k = max(T_k, abs(G_x[len(x) - 1] - G_K_x[len(x) - 1]))
print('T_K =', T_k)

### **5.** Повторите задания пп. 1, 2 и 4 при различных $n$. Постройте график зависимости $T_K$ от $n$. Необходимо взять не менее $10$ различных значений $n$. Графики функций $\hat G_K$ и $G$ (см. п.3) можно построить при одном фиксированном значении $n$ на Ваш выбор.

In [ ]:
def T_k_n(n):
    K = n ** 2
    lambda_ = 1
    samples = []
    sorted_samples = []
    n_order_statistics = []
    for i in range(K):
        samples.append(expon.rvs(scale=1 / lambda_, size=n))

    for i in range(K):
        sorted_samples.append(sorted(samples[i]))
        n_order_statistics.append(sorted_samples[i][-1])

    G_K_x = []
    G_x = []
    x = sorted(n_order_statistics)
    b = np.log(n)

    for i in range(len(x)):
        x[i] -= b

    for i in range(len(x)):
        G_K_x.append(i / K)
        G_x.append(np.exp(-np.exp(-x[i])))

    T_k = 0
    for i in range(len(x) - 1):
        T_k = max(T_k, abs(G_x[i] - G_K_x[i]), abs(G_x[i] - G_K_x[i + 1]))
    T_k = max(T_k, abs(G_x[len(x) - 1] - G_K_x[len(x) - 1]))

    return T_k

In [ ]:
N = np.arange(10, 150, 5)
T_K = []
num_of_mesures = 10
for n in N:
    T = 0
    for i in range(num_of_mesures):
        T += T_k_n(n)
    T_K.append(T / num_of_mesures)

In [ ]:
plt.figure(figsize=(9, 6))
plt.plot(N, T_K)
plt.plot(N, T_K, '.', color="red")
plt.title('график зависимости $T_K$ от $n$')
plt.xlabel('n', fontsize=15)
plt.ylabel('$T_K$', fontsize=15)
plt.grid()
plt.show()

## ***Задача 2: квадратичные формы гауссовских случайных векторов с идемподентными матрицами (4 балла)***

### **1.** Зафиксируйте $n \in \mathbb{N}$ и натуральное $r$ от $2$ до $(n - 1)$.

In [ ]:
n = 100
r = 80

### **2.** Выберите симметричную идемпотентную матрицу $A$ ранга $r$ (то есть симметричную матрицу ранга $r$, удовлетворяющую равенству $A^2 = A$). Матрица не должна быть диагональной. В замечании 2 описан один из способов получения такой матрицы, но Вы можете воспользоваться и другими на Ваше усмотрение.

In [ ]:
columns = []

for i in range(r):
    vec = []
    for j in range(n):
        vec.append(uniform.rvs(loc=-1000, scale=2000))
    vec_copy = vec.copy()
    for j in range(len(columns)):
        vec -= np.dot(vec_copy, columns[j]) * columns[j]
    columns.append(vec / np.linalg.norm(vec))

columns[0] = np.array(columns[0])

Q = np.matrix(columns)
Q = Q.T
A = Q @ Q.T

### **3.** Сгенерируйте $K = n^2$ случайных векторов $X_1, X_2, ... , X_K \in \mathbb{R}^n$ из нормального распределения $\mathcal{N}(0, I_n)$. 

In [ ]:
K = n ** 2
mu, sigma = 0, 1
samples = []
for i in range(K):
    sample = np.random.normal(mu, sigma, n)
    samples.append(sample)

### **4.** По набору наблюдений $X_1^\top AX_1, ..., X_K^\top AX_K$ постройте гистограмму. На тот же рисунок добавьте график плотности распределения $\chi^2(r)$.

In [ ]:
values = []
for i in range(K):
    values.append((samples[i].T @ A @ samples[i]).item())

fig, ax = plt.subplots(figsize=(15, 10))
X = np.arange(0, 150, 0.1)
plt.plot(X, chi2.pdf(X, r))
plt.hist(values, bins=100, density=True)

plt.title('Histogram')
plt.xlabel('$x$', fontsize=15)
plt.ylabel('$p_{\chi^2(r)}$', fontsize=15)
plt.legend(labels=['плотность $\chi^2(r)$', 'гистограмма наблюдений'])
plt.grid()
plt.show()

### **5.** убедитесь, что эмперическая функция распределения \begin{equation}
    \hat F_K(t) = \cfrac{1}{K}\sum\limits_{k=1}^{K} \mathbb{1}\left( X_k^\top AX_k \le t \right), \;\;\; t \ge 0, \end{equation} 
### близка к функции распределения $\chi^2(r)$. Для этого вычислите \begin{equation} T_K = \operatorname{sup}\limits_{t \ge 0}\left| \hat F_K(t) - G_r(t)\right|, \end{equation} где $G_r(t)$ - функция распределения $\chi^2(r)$ (доступна в библиотеке scipy). Обратите внимание, что $\hat F_K$ -- кусочно постоянная функция, а $G_r$ - монотонная. Поэтому значение статистики $T_K$ можно найти точно.

In [ ]:
F_x = []
x = sorted(values)

for i in range(len(x)):
    F_x.append(i / K)

T_k = 0
for i in range(len(x) - 1):
    T_k = max(T_k, abs(F_x[i] - chi2.cdf(x[i], r)), abs(F_x[i + 1] - chi2.cdf(x[i], r)))
T_k = max(T_k, abs(F_x[len(x) - 1] - chi2.cdf(x[len(x) - 1], r)))

print("T_K = ", T_k)

### **6.** Повторите задания пп. 1, 2, 3, 5 при различных значениях $r$ и $n$. При каждом $r$ постройте график зависимости $T_K$ от $n$. Убедитесь, что значение $T_K$ стремится к нулю с ростом $n$. Необходимо выбрать не менее трёх различных значений $r$ и не менее $10$ различных значений $n$. Гистограмму (см. п. 4) можно построить при одном фиксированном значении $n$ на Ваш выбор.

In [ ]:
def T_k(n, r):
    columns = []

    for i in range(r):
        vec = []
        for j in range(n):
            vec.append(uniform.rvs(loc=-1000, scale=2000))
        vec_copy = vec.copy()
        for j in range(len(columns)):
            vec -= np.dot(vec_copy, columns[j]) * columns[j]
        columns.append(vec / np.linalg.norm(vec))

    columns[0] = np.array(columns[0])

    Q = np.matrix(columns)
    Q = Q.T
    A = Q @ Q.T

    K = n ** 2
    mu, sigma = 0, 1
    samples = []
    for i in range(K):
        sample = np.random.normal(mu, sigma, n)
        samples.append(sample)

    values = []
    for i in range(K):
        values.append((samples[i].T @ A @ samples[i]).item())

    F_x = []
    x = sorted(values)

    for i in range(len(x)):
        F_x.append(i / K)

    T_k = 0
    for i in range(len(x) - 1):
        T_k = max(T_k, abs(F_x[i] - chi2.cdf(x[i], r)), abs(F_x[i + 1] - chi2.cdf(x[i], r)))
    T_k = max(T_k, abs(F_x[len(x) - 1] - chi2.cdf(x[len(x) - 1], r)))

    return T_k

In [ ]:
def T_K_list(r, N):
    T_K = []
    num_of_mesures = 10
    for n in N:
        T = 0
        for i in range(num_of_mesures):
            T += T_k(n, r)
        T_K.append(T / num_of_mesures)
    return T_K


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 15))

N = np.arange(10, 100, 5)
list = T_K_list(10, N)
axes[0, 0].plot(N, list)
axes[0, 0].plot(N, list, '.', color="red")
axes[0, 0].set_title("r = 10")
axes[0, 0].set_xlabel("n")
axes[0, 0].set_ylabel('$T_K$')
axes[0, 0].grid()

N = np.arange(30, 100, 5)
list = T_K_list(30, N)
axes[0, 1].plot(N, list)
axes[0, 1].plot(N, list, '.', color="red")
axes[0, 1].set_title("r = 30")
axes[0, 1].set_xlabel("n")
axes[0, 1].set_ylabel('$T_K$')
axes[0, 1].grid()


N = np.arange(50, 100, 5)
list = T_K_list(50, N)
axes[1, 0].plot(N, list)
axes[1, 0].plot(N, list, '.', color="red")
axes[1, 0].set_title("r = 50")
axes[1, 0].set_xlabel("n")
axes[1, 0].set_ylabel('$T_K$')
axes[1, 0].grid()


N = np.arange(70, 120, 5)
list = T_K_list(70, N)
axes[1, 1].plot(N, list)
axes[1, 1].plot(N, list, '.', color="red")
axes[1, 1].set_title("r = 70")
axes[1, 1].set_xlabel("n")
axes[1, 1].set_ylabel('$T_K$')
axes[1, 1].grid()

## Задача 3: распределение порядковых статистик (4 балла).


### **1.** Зафиксируйте $n \in \mathbb{N}$ и натуральное $r$ от $1$ до $n$.

In [ ]:
n = 100

### **2.** Сгенерируйте $K = n^2$ выборок $\left( X_1^k, ... , X_n^k \right)$, $1 \le k \le K$, объёма $n$ из равномерного распределения на отрезке $\left[ 0, 1 \right]$. В каждой из $K$ выборок определить $r$-ую порядковую статистику $X_{(r)}^k$.

In [ ]:
r = 70

K = n ** 2
left, right = 0, 1
samples = []
for i in range(K):
    sample = np.random.uniform(left, right, n)
    samples.append(sample)

for i in range(K):
    samples[i] = sorted(samples[i])

r_order_statistics = []
for i in range(K):
    r_order_statistics.append(samples[i][r - 1])


### **3.**  По набору наблюдений $X_{(r)}^1, ... , X_{(r)}^K$ постройте гистограмму. На тот же рисунок добавьте график плотности распределения $\operatorname{Beta}(r, n + 1 - r)$.

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
X = np.arange(0.4, 1.0, 0.01)
plt.plot(X, beta.pdf(X, r, n + 1 - r))

plt.title('гистограмма', fontsize=15)
plt.xlabel('x')
plt.xlabel('$p(x)$')
plt.hist(r_order_statistics, bins=100, density=True)
plt.grid()

plt.show()

### **4.** Убедитесь, что эмпирическая функция распределения \begin{equation} \hat F_K(t) = \cfrac{1}{K} \sum\limits_{k = 1}^{K}\mathbb{1}\left( X_{(r)}^k \le t \right), \;\;\; t \in [0, 1] \end{equation} близка к функции распределения $\operatorname{Beta}(r, n + 1 − r)$. Для этого вычислите \begin{equation} T_K = \operatorname{sup}_{t \ge 0} \left| \hat F_K(t) - G_{r, n+1-r}(t) \right|, \end{equation} где $G_{r, n + 1 - r}$  - функция распределения $\operatorname{Beta}(r, n + 1 - r)$ (доступна в библиотеке scipy). Обратите внимание, что $\hat F_K$ - кусочно постоянная функция, а $G_{r, n+1-r}$ - монотонная. Поэтому значение статистики $T_K$ можно найти точно.

In [ ]:
F_x = []
x = sorted(r_order_statistics)

for i in range(len(x)):
    F_x.append(i / K)

T_k = 0
for i in range(len(x) - 1):
    T_k = max(T_k, abs(F_x[i] - beta.cdf(x[i], r, n + 1 - r)), abs(F_x[i + 1] - beta.cdf(x[i], r, n + 1 - r)))
T_k = max(T_k, abs(F_x[len(x) - 1] - beta.cdf(x[len(x) - 1], r, n + 1 - r)))

print("T_k = ", T_k)

### **5.** Повторите задания пп. 1, 2, 4 при различных значениях $r$ и $n$. При каждом $r$ постройте график зависимости $T_K$ от $n$. Убедитесь, что значение $T_K$ стремится к нулю с ростом $r$. Необходимо выбрать не менее трех различных значений $r$ и не менее $10$ различных значений $n$. Гистограмму (см. п. 3) можно построить при одном фиксированном значении $n$ на Ваш выбор.

In [ ]:
def T_K(r, n):
    K = n ** 2
    left, right = 0, 1
    samples = []
    for i in range(K):
        sample = np.random.uniform(left, right, n)
        samples.append(sample)

    for i in range(K):
        samples[i] = sorted(samples[i])

    r_order_statistics = []
    for i in range(K):
        r_order_statistics.append(samples[i][r - 1])

    F_x = []
    x = sorted(r_order_statistics)

    for i in range(len(x)):
        F_x.append(i / K)

    T_k = 0
    for i in range(len(x) - 1):
        T_k = max(T_k, abs(F_x[i] - beta.cdf(x[i], r, n + 1 - r)), abs(F_x[i + 1] - beta.cdf(x[i], r, n + 1 - r)))
    T_k = max(T_k, abs(F_x[len(x) - 1] - beta.cdf(x[len(x) - 1], r, n + 1 - r)))

    return T_k

In [ ]:
def T_K_list(r, N):
    T_k = []
    num_of_mesures = 10
    for n in N:
        T = 0
        for i in range(num_of_mesures):
            T += T_K(r, n)
        T_k.append(T / num_of_mesures)
    return T_k


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 15))

N = np.arange(10, 100, 5)
list = T_K_list(10, N)
axes[0, 0].plot(N, list)
axes[0, 0].plot(N, list, '.', color="red")
axes[0, 0].set_title("r = 10")
axes[0, 0].set_xlabel("n")
axes[0, 0].set_ylabel('$T_K$')
axes[0, 0].grid()
N = np.arange(30, 100, 5)
list = T_K_list(30, N)
axes[0, 1].plot(N, list)
axes[0, 1].plot(N, list, '.', color="red")
axes[0, 1].set_title("r = 30")
axes[0, 1].set_xlabel("n")
axes[0, 1].set_ylabel('$T_K$')
axes[0, 1].grid()


N = np.arange(50, 100, 5)
list = T_K_list(50, N)
axes[1, 0].plot(N, list)
axes[1, 0].plot(N, list, '.', color="red")
axes[1, 0].set_title("r = 50")
axes[1, 0].set_xlabel("n")
axes[1, 0].set_ylabel('$T_K$')
axes[1, 0].grid()


N = np.arange(70, 120, 5)
list = T_K_list(70, N)
axes[1, 1].plot(N, list)
axes[1, 1].plot(N, list, '.', color="red")
axes[1, 1].set_title("r = 70")
axes[1, 1].set_xlabel("n")
axes[1, 1].set_ylabel('$T_K$')
axes[1, 1].grid()
